# IFRS 9 ECL Calculation

**Objective**:
- Load my **best PD model** (e.g., tuned XGBoost) to get Probability of Default for each loan.
- Assume or derive key components:
  - **EAD** (Exposure at Default)
  - **LGD** (Loss Given Default)
- Perform a **Stage 1 ECL** calculation: 
  \[
  ECL = PD \times LGD \times EAD
  \]
- (Optional) Introduce a simple macro scenario (e.g., stressed PD = PD × 1.2) to illustrate IFRS 9 forward-looking approach.

**Context**:
In IFRS 9, banks must incorporate forward-looking information and stage the assets (Stage 1, 2, 3). For this simplified demonstration:
- I'll treat all loans as Stage 1 (one-year ECL).
- I'll not do advanced staging triggers or multi-year transitions here.
- I'll show how model-based PD flows into an ECL figure for each loan or the whole portfolio.


### Load Data & Best Model

In [5]:
import pandas as pd
import numpy as np
import joblib

# Load the cleaned dataset (the same one used for final modeling)
df_cleaned = pd.read_csv("../data/credit_risk_dataset_cleaned.csv")
print("Cleaned dataset shape:", df_cleaned.shape)

# If your best model was the tuned XGBoost, load it from a pickle or joblib if you saved it
# Or re-instantiate & re-fit quickly if needed (less ideal).
import joblib

best_xgb = joblib.load("../models/best_xgb.pkl")  # Example path if you saved the model
# If you haven't saved it, you can re-run the training code or do it right here.

df_cleaned.head()

Cleaned dataset shape: (29465, 13)


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_to_income_ratio
0,22.0,59000.0,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3,0.593210
1,21.0,9600.0,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2,0.104156
2,25.0,9600.0,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3,0.572857
3,23.0,65500.0,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2,0.534343
4,24.0,54400.0,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4,0.643371
